Iván Vázquez Martínez

# Examen OPI Analytics

## Sección B

## 1. Procesamiento de los datos

### a. ¿Cuántos registros hay?




In [1]:
import findspark
findspark.init(r"C:\apache-spark\spark-3.0.1-bin-hadoop2.7")
findspark.find()

'C:\\apache-spark\\spark-3.0.1-bin-hadoop2.7'

In [2]:
import sys
sys.path.insert(0, '.')
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
from commons.Utils import Utils

conf = SparkConf().setAppName("airports").setMaster("local[*]")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

In [3]:
path = r"C:\Users\sephc\python-spark-tutorial\profeco_all_data.csv"

fulldf = spark.read.csv(path, header=True, inferSchema= True)

In [5]:
fulldf.show(1)

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
+-------------------

In [6]:
fulldf.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [7]:
print("El dataframe contiene {:,} registros ".format(fulldf.count()))

El dataframe contiene 62,530,715 registros 


### b. ¿Cuántas categorías?

In [8]:
print("Se tienen {:,} categorias diferentes.".format(fulldf.select("categoria").distinct().count()))

Se tienen 42 categorias diferentes.


### c. ¿Cuántas cadenas comerciales están siendo monitoreadas?

In [9]:
fulldf.select("cadenaComercial").distinct().count()

706

### d. ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?

La calidad la podemos determinar checando las siguientes premisas:
1. Existencia de datos
2. Consistencia en los datos
3. Precisión en los datos
4. Integridad 
5. Validez

Detecté que la longitud y latitud deberían tener tipo de dato numérico, no cadena. Los nombres de los municipios tienen errores y existen datos faltantes (valores null en los estados).
  

### e. ¿Cuáles son los productos más monitoreados en cada entidad?

In [5]:
from pyspark.sql.functions import countDistinct
fulldf.groupBy('estado','producto').count().show(truncate=False)

+-------------------------------+------------------------------+-----+
|estado                         |producto                      |count|
+-------------------------------+------------------------------+-----+
|MÉXICO                         |TINTE PARA EL CABELLO         |44007|
|MÉXICO                         |TELEVISORES                   |29702|
|MÉXICO                         |ACELGA                        |7691 |
|MÉXICO                         |QUESO. COTIJA                 |4414 |
|DISTRITO FEDERAL               |AZUCAR                        |18078|
|MÉXICO                         |DESENFRIOL-ITO                |642  |
|JALISCO                        |ARROZ                         |11735|
|OAXACA                         |PEDIALYTE. ELECTROLITOS ORALES|302  |
|TLAXCALA                       |AGUA SIN GAS                  |14505|
|VERACRUZ DE IGNACIO DE LA LLAVE|TOMATE                        |652  |
|MICHOACÁN DE OCAMPO            |PAN DE CAJA                   |13003|
|YUCAT

### f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

Agrupamos por "cadenaComercial", hacemos el conteo y usamos collect para obtener un arreglo:

In [6]:
arreglo = fulldf.groupBy("cadenaComercial").count().collect()

Obtenemos el máximo y de aquí obtenemos la cadena comercial con mayor variedad:

In [9]:
maximo = max(arreglo , key = lambda x : x[1])

print(maximo[0],maximo[1])

WAL-MART 8643133


## 2. Análisis exploratorio

### a. Genera una canasta de productos básicos que te permita comparar los precios geográfica y temporalmente. Justifica tu elección y procedimiento

Segun la pagina (https://www.gob.mx/canastabasica) del gobierno, la canasta básica está conformada por 40 productos:

<div>
<img src="https://www.gob.mx/cms/uploads/image/file/508238/CANASTA_B_SICA_04_BECA_BIENESTAR_EDUCACI_N_MEDIA_SUPERIOR-32_04_BECA_BIENESTAR_EDUCACI_N_MEDIA_SUPERIOR-32__1_.jpg" width="600"/>
</div>

Con estos productos podemos crear una lista, para cada elemento de esta lista analizamos el precio en cada localidad y comparamos el precio para diferentes fechas. De esta manera podemos realizar graficos para cada producto y para cada localidad del precio vs tiempo.

Consideremos los primero 10 productos de la canasta:

In [10]:
canasta = ["MAIZ", "FRIJOL","ARROZ","AZUCAR","HARINA","ACEITE","ATUN","SARDINA","LECHE","CHILE" ]

Creamos un arreglo con los datos que nos interesan:

In [11]:
canastadf = fulldf.groupBy(["estado","producto","precio"]).count().collect()

### b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

Creamos un dataframe con los datos del arreglo anterior:

In [15]:
import pandas as pd

canasta_df = pd.DataFrame(canastadf, columns = ["estado","producto","precio","count"])
canasta_df.head()

,estado,producto,precio,count
0,MÉXICO,GALLETAS,27.00,428
1,MÉXICO,REFRESCO,8.40,110
2,MÉXICO,LECHE EN POLVO,34.95,105
3,DISTRITO FEDERAL,LECHE ULTRAPASTEURIZADA,12.70,485
4,DISTRITO FEDERAL,CARNE TERNERA,99.90,99


Agrupamos por estado , tomamos la cuenta y filtramos la columna precio:

In [23]:
estado_precio = canasta_df.groupby("estado")
estado_precio.head()


,estado,producto,precio,count
0,MÉXICO,GALLETAS,27.00,428
1,MÉXICO,REFRESCO,8.40,110
2,MÉXICO,LECHE EN POLVO,34.95,105
3,DISTRITO FEDERAL,LECHE ULTRAPASTEURIZADA,12.70,485
4,DISTRITO FEDERAL,CARNE TERNERA,99.90,99
...,...,...,...,...
243320,"ESQ. SUR 125""",PAN BLANCO BOLILLO,1.00,1
311633,"ESQ. SUR 125""",PAN BLANCO BOLILLO,1.50,128
1002701,"ESQ. SUR 125""",TORTILLA DE MAIZ,11.00,2
2148146,estado,producto,NaN,20


In [65]:
estado_pr = estado_precio.head()
grup = estado_pr.groupby("estado").sum()

print("Con estos datos deducimos que la ciudad más cara se encuentra en el estado de: ",grup.sort_values("precio",ascending=False).index[0])
print("Y la ciudad más barata se encuentra en el estado de: ",grup.sort_values("precio",ascending=False).index[-3])

Con estos datos deducimos que la ciudad más cara se encuentra en el estado de:  QUERÉTARO
Y la ciudad más barata se encuentra en el estado de:  GUANAJUATO


### c. ¿Hay algún patrón estacional entre años?

Seleccionamos las columnas utiles:

In [66]:
filtro = fulldf["producto","estado","fechaRegistro","precio"]

Creamos un arreglo agrupado por fecha de registro y un contador:

In [68]:
filtro2 = filtro["fechaRegistro","precio"]
arreglo = filtro2.groupBy(["fechaRegistro"]).count().collect()

Py4JJavaError: An error occurred while calling o83.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at scala.reflect.ClassTag$GenericClassTag.newArray(ClassTag.scala:151)
	at scala.collection.TraversableOnce.toArray(TraversableOnce.scala:290)
	at scala.collection.TraversableOnce.toArray$(TraversableOnce.scala:288)
	at scala.collection.AbstractTraversable.toArray(Traversable.scala:108)
	at org.apache.spark.sql.execution.SparkPlan.executeCollect(SparkPlan.scala:388)
	at org.apache.spark.sql.Dataset.$anonfun$collectToPython$1(Dataset.scala:3450)
	at org.apache.spark.sql.Dataset$$Lambda$2878/945736984.apply(Unknown Source)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3618)
	at org.apache.spark.sql.Dataset$$Lambda$1337/70087060.apply(Unknown Source)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:100)
	at org.apache.spark.sql.execution.S

In [ ]:
dftemporal = pd.DataFrame(arreglo,columns = ["fechaRegistro","count"])

#De las fechas de registro podemos extraer mes y año:
dftemporal["mes"] = dftemporal["fechaRegistro"].apply(lambda x: x[:4])
dftemporal["año"] = dftemporal["fechaRegistro"].apply(lambda x: x[5:7])


### d. ¿Cuál es el estado más caro y en qué mes?

Según el inciso b , el estado más caro es Queretaro

### e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con estos datos?

Ya que los precios de los productos varían de estado a estado, se encontrarían muchas discrepancias a la hora de hacer un
analisis temporal.

## 3. Visualización

### a. Genera un mapa que nos permita identificar la oferta de categorías en la zona metropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se darán puntos extra si el mapa es interactivo